# Extraer tablas de PDF Libro Rojo de las Aves

In [1]:
#pip install pdfplumber

In [2]:
import pdfplumber
import pandas as pd
import csv
import numpy as np

In [3]:
# Cargo los PDF del LIBRO ROJO DE LAS AVES DE ESPAÑA 2021 Para obtener los estados de conservación de las aves 
pdf_path = ["C:/Users/monic/Downloads/ilovepdf_split-range (2)/LIbro-Rojo-web-3_01-69-86-1.pdf",
            "C:/Users/monic/Downloads/ilovepdf_split-range (2)/LIbro-Rojo-web-3_01-69-86-2.pdf",
            "C:/Users/monic/Downloads/ilovepdf_split-range (2)/LIbro-Rojo-web-3_01-69-86-3.pdf",
            "C:/Users/monic/Downloads/ilovepdf_split-range (2)/LIbro-Rojo-web-3_01-69-86-4.pdf",
            "C:/Users/monic/Downloads/ilovepdf_split-range (2)/LIbro-Rojo-web-3_01-69-86-5.pdf",
            "C:/Users/monic/Downloads/ilovepdf_split-range (2)/LIbro-Rojo-web-3_01-69-86-6.pdf",
            "C:/Users/monic/Downloads/ilovepdf_split-range (2)/LIbro-Rojo-web-3_01-69-86-7.pdf",
            "C:/Users/monic/Downloads/ilovepdf_split-range (2)/LIbro-Rojo-web-3_01-69-86-8.pdf",
            "C:/Users/monic/Downloads/ilovepdf_split-range (2)/LIbro-Rojo-web-3_01-69-86-9.pdf",
            "C:/Users/monic/Downloads/ilovepdf_split-range (2)/LIbro-Rojo-web-3_01-69-86-10.pdf",
            "C:/Users/monic/Downloads/ilovepdf_split-range (2)/LIbro-Rojo-web-3_01-69-86-11.pdf",
            "C:/Users/monic/Downloads/ilovepdf_split-range (2)/LIbro-Rojo-web-3_01-69-86-12.pdf",
            "C:/Users/monic/Downloads/ilovepdf_split-range (2)/LIbro-Rojo-web-3_01-69-86-13.pdf",
            "C:/Users/monic/Downloads/ilovepdf_split-range (2)/LIbro-Rojo-web-3_01-69-86-14.pdf",
            "C:/Users/monic/Downloads/ilovepdf_split-range (2)/LIbro-Rojo-web-3_01-69-86-15.pdf",
            "C:/Users/monic/Downloads/ilovepdf_split-range (2)/LIbro-Rojo-web-3_01-69-86-16.pdf",
            "C:/Users/monic/Downloads/ilovepdf_split-range (2)/LIbro-Rojo-web-3_01-69-86-17.pdf",
            "C:/Users/monic/Downloads/ilovepdf_split-range (2)/LIbro-Rojo-web-3_01-69-86-18.pdf"
            ]

all_extracted_dfs = [] # Aquí se guradan los DF que se van extrayendo de los PDFs.

for path in pdf_path:

    with pdfplumber.open(path) as pdf:
        for page in pdf.pages:
        # Definimos como tiene que buscar las tablas para luego convertilas en DFs
            table_settings = { 
                "vertical_strategy": "lines",
                "horizontal_strategy": "lines"
            }

        tables = page.extract_tables() 

        for table_data in tables:
            if table_data:
                df = pd.DataFrame(table_data[1:], columns=table_data[0])
                all_extracted_dfs.append(df) #Convierte las tablas en DFs y los añade a la lista de DFs


## Compruebo i elimino aquellos DF que tienen pocas columnas 

In [4]:
# Cuantos DF se han extraido? 
df_filtrados =[] #Aquí se guardan los DFs con información valida

print(f"\nNumero total de DataFrames extraidos: {len(all_extracted_dfs)}")

# Pido que me haga un resumen porque algunos DF no contienen información de los estatus, solo de las páginas del PDF y quiero eliminar estos DF sin informació.
for i, df in enumerate(all_extracted_dfs):
    print(f'\nResumen del Dataframe numero {i + 1}')
    print(df.head(3))
    if df.shape[1] < 5:
        print(f'\n Dataframe {i +1} con dos columnas, no lo incluimos en la concatenación')
    else:
        print(f'\nDataframe {i+1} con más de dos columnas, lo añadimos a la lista de dataframes filtrados')
        df_filtrados.append(df)
        

# Comprobar y concatenar los DFs filtrados      
final_combined_df = pd.DataFrame()

try: 
    final_combined_df = pd.concat(df_filtrados, ignore_index=True)
except Exception as e: 
    print(f'Error al concatenar DataFrame: {e}')
    
#Cambio de nombres    
final_combined_df.rename(columns = {
    'NOMBRE CIENTÍFICO':'Scientific name'}, inplace=True)

# Elimino esta columna porque es una columna vacia, ya que en el PDF original esta información vien en vorma de icono que pdfplumber no sabe interpretar
final_combined_df.drop(['EVOLUCIÓN\nCATEGORÍA\n2004-2021'], axis=1, inplace=True)
final_combined_df


Numero total de DataFrames extraidos: 36

Resumen del Dataframe numero 1
  68 LIBRO ROJO DE LAS AVES DE ESPAÑA
0                                None

 Dataframe 1 con dos columnas, no lo incluimos en la concatenación

Resumen del Dataframe numero 2
  EVOLUCIÓN\nCATEGORÍA\n2004-2021       NOMBRE COMÚN NOMBRE CIENTÍFICO  \
0                                  Abejaruco europeo   Merops apiaster   
1                                    Abejero europeo   Pernis apivorus   
2                                     Abubilla común       Upupa epops   

  LR2004\n(*Canarias) LISTA ROJA 2021\nInvern./Migrat.  \
0                  NE                                    
1                  LC                                    
2                  NE                                    

  LISTA ROJA 2021\nReproductoras\n(*Canarias)  
0                                          LC  
1                                          NT  
2                                          LC  

Dataframe 2 con más de dos 

,NOMBRE COMÚN,Scientific name,LR2004\n(*Canarias),LISTA ROJA 2021\nInvern./Migrat.,LISTA ROJA 2021\nReproductoras\n(*Canarias)
0,Abejaruco europeo,Merops apiaster,NE,,LC
1,Abejero europeo,Pernis apivorus,LC,,NT
2,Abubilla común,Upupa epops,NE,,LC
3,Acentor alpino,Prunella collaris,NE,,NT
4,Acentor común,Prunella modularis,NE,,LC
...,...,...,...,...,...
368,Zarcero políglota,Hippolais polyglotta,NE,,LC
369,Zorzal alirrojo,Turdus iliacus,NE,DD,
370,Zorzal charlo,Turdus viscivorus,NE,,LC
371,Zorzal común,Turdus philomelos,NE,,LC


## Quiero quedarme unicamente con aquellas especies que aparecen en mi DF 'madre'

In [5]:
# Importar el DF
censo = pd.read_csv(r'C:/Users/monic/OneDrive/Desktop/It Academmy - Data Analysis/Projecte/Python archibos buenos/censo_aves_temperaturas_pesquerias.csv', delimiter=',', quotechar='"')

#Cambio el nombre cientifico del cormoran mñudo a su nombre cientifico correcto
censo['Scientific name'] = censo['Scientific name'].replace('Phalacrocorax aristotelis desmarestii', 'Phalacrocorax aristotelis')
censo['Scientific name'] = censo['Scientific name'].replace('Ichthyaetus melanocephalus', 'Larus melanocephalus')
censo['Scientific name'] = censo['Scientific name'].replace('Ichthyaetus audouinii', 'Larus audouinii')

# Cambio el nombre cientifico de la gaviota reidora a su nombre cientifico correcto.
censo['Scientific name'] = censo['Scientific name'].replace('Larus ridibunus', 'Chroicocephalus ridibundus')

# Hacer el merge
censo_2 = censo.merge(final_combined_df, on ='Scientific name', how='left')
censo_2

,date,speciesname,Scientific name,year,yday,fecha,fecha_formateada,month,nombre_mes,count_aves,...,Arts menors en el Mediterrani,Encerclament al Mediterrani,Encerclament tonyina vermella al Mediterrani,Palangre de fons al Mediterrani,Palangre de superficie al Mediterrani,Total CAT,NOMBRE COMÚN,LR2004\n(*Canarias),LISTA ROJA 2021\nInvern./Migrat.,LISTA ROJA 2021\nReproductoras\n(*Canarias)
0,38830,Balearic Shearwater,Puffinus mauretanicus,2006,113,23/04/2006,2006-04-23,4,Abril,7,...,771.0,109.0,6.0,68.0,11.0,1311.0,Pardela balear,CR,,CR
1,38830,Balearic-/Yelkouan Shearwater,Puffinus sp.,2006,113,23/04/2006,2006-04-23,4,Abril,3,...,771.0,109.0,6.0,68.0,11.0,1311.0,NaN,NaN,NaN,NaN
2,38830,Cormorant,Phalacrocorax carbo,2006,113,23/04/2006,2006-04-23,4,Abril,1,...,771.0,109.0,6.0,68.0,11.0,1311.0,Cormorán grande,NE,LC,LC
3,38830,Gannet,Morus bassanus,2006,113,23/04/2006,2006-04-23,4,Abril,1,...,771.0,109.0,6.0,68.0,11.0,1311.0,Alcatraz atlántico,NE,LC,
4,38830,Mediterranean Gull,Larus melanocephalus,2006,113,23/04/2006,2006-04-23,4,Abril,1,...,771.0,109.0,6.0,68.0,11.0,1311.0,Gaviota cabecinegra,NE,NT,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2105,45780,Sandwich Tern,Thalasseus sandvicensis,2025,123,03/05/2025,2025-05-03,5,Mayo,14,...,314.0,52.0,4.0,13.0,9.0,583.0,Charrán patinegro,NT,,VU
2106,45780,Scopoli's Shearwater,Calonectris diomedea,2025,123,03/05/2025,2025-05-03,5,Mayo,276,...,314.0,52.0,4.0,13.0,9.0,583.0,Pardela cenicienta mediterránea,EN,,EN
2107,45780,Shag,Phalacrocorax aristotelis,2025,123,03/05/2025,2025-05-03,5,Mayo,40,...,314.0,52.0,4.0,13.0,9.0,583.0,Cormorán moñudo,EN/VU,,VU(EN/VU)
2108,45780,Yelkouan Shearwater,Puffinus yelkouan,2025,123,03/05/2025,2025-05-03,5,Mayo,204,...,314.0,52.0,4.0,13.0,9.0,583.0,Pardela mediterránea,NE,,VU


##  Añadir la evolución de la categoria

In [6]:
#Genero el diccionario manualmente con la info del PDF
evolucion_status = {
    'Stercorarius parasiticus':'No Evaluada',
    'Stercorarius pomarinus':'No Evaluada',
    'Stercorarius skua': 'No Evaluada',
    'Puffinus mauretanicus': 'Sin Variaciones',
    'Puffinus yelkouan':'Desfavorable',
    'Calonectris diomedea' : 'Sin Variaciones',
    'Ardenna grisea':'No Evaluada',
    'Hydrobates pelagicus': 'Desfavorable',
    'Fratercula arctica':'Datos Insuficientes',
    'Alca torda':'Datos Insuficientes',
    'Phalacrocorax aristotelis':'Sin Variaciones',
    'Phalacrocorax carbo':'Favorable',
    'Morus bassanus': 'Favorable',
    'Sterna hirundo':'Sin Variaciones',
    'Thalasseus sandvicensis':'Desfavorable',
    'Chroicocephalus ridibundus':'Favorable',
    'Larus audouinii':'Sin Variaciones',
    'Rissa tridactyla': 'Desfavorable',
    'Larus fuscus': 'Sin Variaciones',
    'Hydrocoloeus minutus': 'No Evaluada',
    'Larus melanocephalus': 'Desfavorable',
    'Larus michahellis':'Desfavorable'}

#Para comprobar si las especies están en el diccionario
for index, row in censo_2.iterrows():
    scientific_name = row['Scientific name']
    if scientific_name in evolucion_status:
        print(f"'{scientific_name}' encontrado en el diccionario")
    else:
       print(f"'{scientific_name}' no encontrado en el diccionario")
       
censo_2['EVOLUCIÓN CATEGORÍA 2004-2021'] = censo_2['Scientific name'].map(evolucion_status)

censo_2

'Puffinus mauretanicus' encontrado en el diccionario
'Puffinus sp.' no encontrado en el diccionario
'Phalacrocorax carbo' encontrado en el diccionario
'Morus bassanus' encontrado en el diccionario
'Larus melanocephalus' encontrado en el diccionario
'Fratercula arctica' encontrado en el diccionario
'Thalasseus sandvicensis' encontrado en el diccionario
'Calonectris diomedea' encontrado en el diccionario
'Puffinus yelkouan' encontrado en el diccionario
'Larus michahellis' encontrado en el diccionario
'Puffinus mauretanicus' encontrado en el diccionario
'Puffinus sp.' no encontrado en el diccionario
'Phalacrocorax carbo' encontrado en el diccionario
'Morus bassanus' encontrado en el diccionario
'Calonectris diomedea' encontrado en el diccionario
'Phalacrocorax aristotelis' encontrado en el diccionario
'Puffinus yelkouan' encontrado en el diccionario
'Larus michahellis' encontrado en el diccionario
'Chroicocephalus ridibundus' encontrado en el diccionario
'Morus bassanus' encontrado en el 

,date,speciesname,Scientific name,year,yday,fecha,fecha_formateada,month,nombre_mes,count_aves,...,Encerclament al Mediterrani,Encerclament tonyina vermella al Mediterrani,Palangre de fons al Mediterrani,Palangre de superficie al Mediterrani,Total CAT,NOMBRE COMÚN,LR2004\n(*Canarias),LISTA ROJA 2021\nInvern./Migrat.,LISTA ROJA 2021\nReproductoras\n(*Canarias),EVOLUCIÓN CATEGORÍA 2004-2021
0,38830,Balearic Shearwater,Puffinus mauretanicus,2006,113,23/04/2006,2006-04-23,4,Abril,7,...,109.0,6.0,68.0,11.0,1311.0,Pardela balear,CR,,CR,Sin Variaciones
1,38830,Balearic-/Yelkouan Shearwater,Puffinus sp.,2006,113,23/04/2006,2006-04-23,4,Abril,3,...,109.0,6.0,68.0,11.0,1311.0,NaN,NaN,NaN,NaN,NaN
2,38830,Cormorant,Phalacrocorax carbo,2006,113,23/04/2006,2006-04-23,4,Abril,1,...,109.0,6.0,68.0,11.0,1311.0,Cormorán grande,NE,LC,LC,Favorable
3,38830,Gannet,Morus bassanus,2006,113,23/04/2006,2006-04-23,4,Abril,1,...,109.0,6.0,68.0,11.0,1311.0,Alcatraz atlántico,NE,LC,,Favorable
4,38830,Mediterranean Gull,Larus melanocephalus,2006,113,23/04/2006,2006-04-23,4,Abril,1,...,109.0,6.0,68.0,11.0,1311.0,Gaviota cabecinegra,NE,NT,,Desfavorable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2105,45780,Sandwich Tern,Thalasseus sandvicensis,2025,123,03/05/2025,2025-05-03,5,Mayo,14,...,52.0,4.0,13.0,9.0,583.0,Charrán patinegro,NT,,VU,Desfavorable
2106,45780,Scopoli's Shearwater,Calonectris diomedea,2025,123,03/05/2025,2025-05-03,5,Mayo,276,...,52.0,4.0,13.0,9.0,583.0,Pardela cenicienta mediterránea,EN,,EN,Sin Variaciones
2107,45780,Shag,Phalacrocorax aristotelis,2025,123,03/05/2025,2025-05-03,5,Mayo,40,...,52.0,4.0,13.0,9.0,583.0,Cormorán moñudo,EN/VU,,VU(EN/VU),Sin Variaciones
2108,45780,Yelkouan Shearwater,Puffinus yelkouan,2025,123,03/05/2025,2025-05-03,5,Mayo,204,...,52.0,4.0,13.0,9.0,583.0,Pardela mediterránea,NE,,VU,Desfavorable


In [7]:
# Cambio nombres de columnas para que sea más facil trabajar con ellas
censo_2.rename(columns= {
    'LR2004\n(*Canarias)': 'LR2004',
    'LISTA ROJA 2021\nInvern./Migrat.':'LR2021 Invern/Migrat',
    'LISTA ROJA 2021\nReproductoras\n(*Canarias)': 'LR2021 Repro'    
}, inplace=True)

#Añado manualmente información porque se muestra así en mi df del censo Pardela balear/mediterránea' ya que son dos especies 
# dificiles de distinguir pero no quiero eliminar los datos del censo
dic_pardela ={'Puffinus sp.': {
        'NOMBRE COMÚN':'Pardela balear/mediterránea',
        'LR2004':'CR/EN',
        'LR2021 Repro':'CR/VU', 
        'EVOLUCIÓN CATEGORÍA 2004-2021':'Sin Variaciones/Desfavorable'
        }
    }

for scientific_name, values_to_fill in dic_pardela.items():
    mask=censo_2['Scientific name'] == scientific_name
    
    for column, value in values_to_fill.items():
        censo_2.loc[mask, column] = censo_2.loc[mask, column].fillna(value)

# Añadir 'Sin datos' a aquellas casillas que se quedan vacias
censo_2 = censo_2.fillna('Sin datos')
censo_2 = censo_2.replace(r'^\s*$', 'Sin datos', regex=True)

# Genero una nueva columna para poder tener en una única los datos de estado de conservación dependiendo si la especie es migrante o reproductora en España.
censo_2['Estatus Consolidado'] = np.where(
    censo_2['LR2021 Repro'] == 'Sin datos',  # Condición: si el estatus de reproductoras es 'Sin datos'
    censo_2['LR2021 Invern/Migrat'],          # Si es True, toma el valor de la columna de invernantes
    censo_2['LR2021 Repro']                   # Si es False, toma el valor de la columna de reproductoras
)
censo_2

,date,speciesname,Scientific name,year,yday,fecha,fecha_formateada,month,nombre_mes,count_aves,...,Encerclament tonyina vermella al Mediterrani,Palangre de fons al Mediterrani,Palangre de superficie al Mediterrani,Total CAT,NOMBRE COMÚN,LR2004,LR2021 Invern/Migrat,LR2021 Repro,EVOLUCIÓN CATEGORÍA 2004-2021,Estatus Consolidado
0,38830,Balearic Shearwater,Puffinus mauretanicus,2006,113,23/04/2006,2006-04-23,4,Abril,7,...,6.0,68.0,11.0,1311.0,Pardela balear,CR,Sin datos,CR,Sin Variaciones,CR
1,38830,Balearic-/Yelkouan Shearwater,Puffinus sp.,2006,113,23/04/2006,2006-04-23,4,Abril,3,...,6.0,68.0,11.0,1311.0,Pardela balear/mediterránea,CR/EN,Sin datos,CR/VU,Sin Variaciones/Desfavorable,CR/VU
2,38830,Cormorant,Phalacrocorax carbo,2006,113,23/04/2006,2006-04-23,4,Abril,1,...,6.0,68.0,11.0,1311.0,Cormorán grande,NE,LC,LC,Favorable,LC
3,38830,Gannet,Morus bassanus,2006,113,23/04/2006,2006-04-23,4,Abril,1,...,6.0,68.0,11.0,1311.0,Alcatraz atlántico,NE,LC,Sin datos,Favorable,LC
4,38830,Mediterranean Gull,Larus melanocephalus,2006,113,23/04/2006,2006-04-23,4,Abril,1,...,6.0,68.0,11.0,1311.0,Gaviota cabecinegra,NE,NT,Sin datos,Desfavorable,NT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2105,45780,Sandwich Tern,Thalasseus sandvicensis,2025,123,03/05/2025,2025-05-03,5,Mayo,14,...,4.0,13.0,9.0,583.0,Charrán patinegro,NT,Sin datos,VU,Desfavorable,VU
2106,45780,Scopoli's Shearwater,Calonectris diomedea,2025,123,03/05/2025,2025-05-03,5,Mayo,276,...,4.0,13.0,9.0,583.0,Pardela cenicienta mediterránea,EN,Sin datos,EN,Sin Variaciones,EN
2107,45780,Shag,Phalacrocorax aristotelis,2025,123,03/05/2025,2025-05-03,5,Mayo,40,...,4.0,13.0,9.0,583.0,Cormorán moñudo,EN/VU,Sin datos,VU(EN/VU),Sin Variaciones,VU(EN/VU)
2108,45780,Yelkouan Shearwater,Puffinus yelkouan,2025,123,03/05/2025,2025-05-03,5,Mayo,204,...,4.0,13.0,9.0,583.0,Pardela mediterránea,NE,Sin datos,VU,Desfavorable,VU


In [8]:
#Guardar
#censo_2.to_csv('censo_aves_temperaturas_pesquerias_estatus_libro_rojo.csv', index=False)